In [1]:
#! /usr/bin/env python3.5
import tensorflow as tf
import numpy as np

import data.ops
import model.cnn as cnn
import model.rnn as rnn
import model.classifier as classifier

import time
import os
import shutil
import json

flags = tf.app.flags
flags.DEFINE_integer('gpu', 0, 'device to train on [0]')
flags.DEFINE_string('model_def', './hyperparams/test_model.json', 'load hyperparameters from ["model.json"]')
FLAGS = flags.FLAGS
FLAGS._parse_flags()

os.environ['CUDA_VISIBLE_DEVICES'] = str(FLAGS.gpu)

paths = [
    './data/NORMAL_CLASS_REF.TFRecord', 
    './data/OTHER_CLASS_REF.TFRecord',
    './data/ATRIUM_CLASS_REF.TFRecord',
    './data/NOISE_CLASS_REF.TFRecord'
]

In [10]:
tf.reset_default_graph()
batch_size = tf.placeholder_with_default(16, [], name='batch_size')
(input_op, seq_len, label), input_prods = data.ops.get_even_batch_producer(
    paths=paths, batch_size=batch_size)

val_input_op, val_seq_len, val_label = data.ops.get_batch_producer(
    batch_size=batch_size, path='./data/VALIDATION.TFRecord')

cnn_params = {
    'out_dims' : [128, 256, 256],
    'kernel_sizes' : 64,
    'pool_sizes' : 1
}

c = cnn.model(
    seq_len=seq_len, 
    input_op=input_op, 
    model_name='CNN_block',
    **cnn_params)

RESIDUAL_POOL = 3
residual_input = c.output[..., None, :]

for i in range(1, 4):    
    residual_input = tf.contrib.layers.max_pool2d(
        residual_input, 
        kernel_size=[RESIDUAL_POOL, 1], 
        stride=[RESIDUAL_POOL, 1])
    
    c = cnn.model(
        seq_len=seq_len, 
        input_op=residual_input, 
        residual=True, 
        model_name='CNN_block_%d'%i,
        **cnn_params)
    residual_input += c.output

res_out = tf.squeeze(residual_input, axis=2)
a = tf.reduce_mean(res_out, axis=1)
fc = classifier.model(input_op=a, fc_sizes=[16])

logits = fc.logits
pred = fc.pred

MODEL_PATH = '/tmp/balanced/resnet/' + c.name + fc.name
MODEL_EXISTS = os.path.exists(MODEL_PATH)
if MODEL_EXISTS:
    print('Model directory is not empty, removing old files')
    shutil.rmtree(MODEL_PATH)

CNN_block--cnn128x64-256x64-256x64
Tensor("CNN_block/Conv1/dropout/mul:0", shape=(?, ?, 1, 128), dtype=float32)
Tensor("CNN_block/Conv2/dropout/mul:0", shape=(?, ?, 1, 256), dtype=float32)
Tensor("CNN_block/Conv3/dropout/mul:0", shape=(?, ?, 1, 256), dtype=float32)
CNN_block_1--cnn128x64-256x64-256x64
Tensor("CNN_block_1/Conv1/dropout/mul:0", shape=(?, ?, 1, 128), dtype=float32)
Tensor("CNN_block_1/Conv2/dropout/mul:0", shape=(?, ?, 1, 256), dtype=float32)
Tensor("CNN_block_1/Conv3/dropout/mul:0", shape=(?, ?, 1, 256), dtype=float32)
CNN_block_2--cnn128x64-256x64-256x64
Tensor("CNN_block_2/Conv1/dropout/mul:0", shape=(?, ?, 1, 128), dtype=float32)
Tensor("CNN_block_2/Conv2/dropout/mul:0", shape=(?, ?, 1, 256), dtype=float32)
Tensor("CNN_block_2/Conv3/dropout/mul:0", shape=(?, ?, 1, 256), dtype=float32)
CNN_block_3--cnn128x64-256x64-256x64
Tensor("CNN_block_3/Conv1/dropout/mul:0", shape=(?, ?, 1, 128), dtype=float32)
Tensor("CNN_block_3/Conv2/dropout/mul:0", shape=(?, ?, 1, 256), dtype=

In [11]:
def measure_time(op, feed_dict={}, n_times=10):
    with tf.Session() as sess:
        print('Sess started')
        coord = tf.train.Coordinator()
        tf.global_variables_initializer().run()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        
        print('Evaluating')
        for _ in range(n_times):
            t = time.time()
            fetch = sess.run(op, feed_dict)
            print('Eval time:', time.time() - t)
            
        print('Closing threads')
        coord.request_stop()
        coord.join(threads)

        return fetch

# Evaluation

## Samples

## **Confusion matrix**

## **Accuracy operator**

In [12]:
with tf.name_scope('evaluation'):
    with tf.name_scope('one_hot_encoding'):
        y_oh = tf.cast(tf.equal(
            logits, tf.reduce_max(logits, axis=1)[:, None]), tf.float32)

        label_oh = tf.one_hot(label, depth=4)
    with tf.name_scope('confusion_matrix'):
        conf_op = tf.reduce_sum(tf.transpose(
            y_oh[..., None], perm=[0, 2, 1]) * label_oh[..., None],
            axis=0, name='result')

    with tf.name_scope('accuracy'):
        y_tot = tf.reduce_sum(conf_op, axis=0, name='label_class_sum')
        label_tot = tf.reduce_sum(conf_op, axis=1, name='pred_class_sum')
        correct_op = tf.diag_part(conf_op, name='correct_class_sum')
        eps = tf.constant([1e-10] * 4, name='eps')
        acc = tf.reduce_mean(2*correct_op / (y_tot + label_tot + eps), name='result')

In [13]:
class_hist = np.load('./data/class_histogramTRAIN.npy')
with tf.name_scope('loss'):
    #weight = tf.constant([.1, 1, .2, 3])
    weight = tf.constant(1 - np.sqrt(class_hist/class_hist.sum()), name='weights')
    weight = tf.gather(weight, label, name='weight_selector')
    train_loss = tf.losses.softmax_cross_entropy(
        label_oh, logits, weight, scope='weighted_loss')
    unweighted_loss = tf.losses.softmax_cross_entropy(
        label_oh, logits, scope='unweighted_loss')
    
    l2_loss = tf.reduce_sum([tf.nn.l2_loss(v, name='L2_reg_loss') 
                            for v in tf.trainable_variables()])
    beta = 0.0001
    loss = unweighted_loss + beta * l2_loss
#class_hist, weight

In [14]:
with tf.name_scope('train'):
    TRAIN_STEPS = 8000
    TS = tf.constant(TRAIN_STEPS, name='TRAIN_STEPS')
    global_step = tf.Variable(initial_value=0, trainable=False, name='global_step')
    boundaries = [TS//5, 2*TS//5, 3*TS//5, 4*TS//5]
    values = [0.001, 0.0005, 0.001, 0.0005, 0.00001]
    learning_rate = tf.train.piecewise_constant(global_step, boundaries, values)

    grad_clip = tf.Variable(initial_value=3., trainable=False, name='grad_clip')
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    gvs = optimizer.compute_gradients(loss)
    with tf.name_scope('gradient_clipping'):
        capped_gvs = [(tf.clip_by_value(grad, -grad_clip, grad_clip), var) 
                      for grad, var in gvs]
        
    opt = optimizer.apply_gradients(capped_gvs, global_step)
    
    #opt = optimizer.minimize(1-max)

In [15]:
train_writer = tf.summary.FileWriter(MODEL_PATH, graph=tf.get_default_graph())
sum_ops = []
for v in tf.trainable_variables():
    sum_ops.append(tf.summary.histogram(v.name[:-2], v))
    sum_ops.append(tf.summary.histogram('gradients/'+v.name[:-2], tf.gradients(loss, v)))

sum_ops.append(tf.summary.scalar('weighted_loss', loss))
sum_ops.append(tf.summary.scalar('unweighted_loss', unweighted_loss))
sum_ops.append(tf.summary.scalar('accuracy', acc))
sum_ops.append(tf.summary.image('confusion_matrix', conf_op[None, ..., None], max_outputs=10))
summaries = tf.summary.merge(sum_ops)
eval_summaries = tf.summary.merge([tf.summary.scalar('eval_accuracy', acc), 
    tf.summary.image('confusion_matrix', conf_op[None, ..., None], max_outputs=10)])

In [17]:
saver = tf.train.Saver(keep_checkpoint_every_n_hours=1)

with tf.Session() as sess:
    print('Sess started')
    
    print('Initializing model')
    tf.global_variables_initializer().run()
        
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    print('Training')
    step = 0
    while step < TRAIN_STEPS:
        t = time.time()
        fetch = sess.run([opt, loss, acc, global_step])
        step = fetch[-1]
        print('%6d/%d'%(step, TRAIN_STEPS), 
              'time: %03.2f'%(time.time()-t), 
              'loss: %02.4f'%fetch[1],
              'acc: %1.3f'%fetch[2],
              sep='\t')
        #if step % 50 == 0:
        #    print('Evaluating TRAIN summaries...')
        #    train_writer.add_summary(summaries.eval(), global_step=fetch[-1])
        if step % 100 == 0:
            print('Evaluating VALIDATION summaries...')
            train_writer.add_summary(
                eval_summaries.eval(validation_feed_dict), 
                global_step=fetch[-1])
        if step % 250 == 0:
            print('Saving model...')
            print(saver.save(sess, MODEL_PATH, global_step=fetch[-1]))
    
    print('Ending, closing producer threads')
    coord.request_stop()
    coord.join(threads)

Sess started
Initializing model
Training
     1/8000	time: 19.18	loss: 2.4385	acc: 0.301
     2/8000	time: 11.69	loss: 5.2135	acc: 0.243
     3/8000	time: 11.90	loss: 5.3775	acc: 0.101
     4/8000	time: 30.68	loss: 3.2737	acc: 0.155
     5/8000	time: 11.66	loss: 2.7342	acc: 0.316
     6/8000	time: 28.69	loss: 3.2680	acc: 0.371


KeyboardInterrupt: 

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: producer_./data/NOISE_CLASS_REF.TFRecord/padded_batch_queue/cond/padding_fifo_queue_enqueue = QueueEnqueueV2[Tcomponents=[DT_FLOAT, DT_INT32, DT_INT32], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](producer_./data/NOISE_CLASS_REF.TFRecord/padded_batch_queue/cond/padding_fifo_queue_enqueue/Switch:1, producer_./data/NOISE_CLASS_REF.TFRecord/padded_batch_queue/cond/padding_fifo_queue_enqueue/Switch_1:1, producer_./data/NOISE_CLASS_REF.TFRecord/padded_batch_queue/cond/padding_fifo_queue_enqueue/Switch_2/_181, producer_./data/NOISE_CLASS_REF.TFRecord/padded_batch_queue/cond/padding_fifo_queue_enqueue/Switch_3/_183)]]
	 [[Node: producer_./data/NOISE_CLASS_REF.TFRecord/padded_batch_queue/cond/padding_fifo_queue_enqueue/_186 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="